# Note: kindly check runs/exp_default for plots and colored ply files at each training epoch and runs/exp_default_exports_validation for final validation plots and colored ply files

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

PROJECT_ROOT = "/content/drive/MyDrive/kpconv_abc"
print("Project ready at:", PROJECT_ROOT)


Mounted at /content/drive
Project ready at: /content/drive/MyDrive/kpconv_abc


Fix compiling problems

In [ ]:
%pip -q install -U pip
%pip -q install "numpy==1.26.4" "scipy==1.11.4" "scikit-learn==1.4.2" plyfile
!apt-get -qq update
!apt-get -qq install -y build-essential cmake

import sys, numpy as np
print("Python:", sys.version.split()[0], "| NumPy:", np.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.4.2 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.11.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
cuml-cu12 25.6.0 requires scikit-learn>=1.5, but you have scikit-learn 1.4.2 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9

In [ ]:
%pip -q install -U plyfile

In [ ]:
# Toolchain
!apt-get -qq update
!apt-get -qq install -y build-essential cmake

# Clone or update the repo
import os, glob
if not os.path.isdir("/content/KPConv-PyTorch"):
    !git clone --depth 1 https://github.com/HuguesTHOMAS/KPConv-PyTorch.git /content/KPConv-PyTorch
else:
    %cd /content/KPConv-PyTorch
    !git pull

# Remove any stale .so (both in clone and your project) to avoid ABI mismatches
PROJECT_ROOT = "/content/drive/MyDrive/kpconv_abc"
for p in glob.glob("/content/KPConv-PyTorch/cpp_wrappers/**/*.so", recursive=True):
    try: os.remove(p)
    except: pass
for p in glob.glob(f"{PROJECT_ROOT}/cpp_wrappers/**/*.so", recursive=True):
    try: os.remove(p)
    except: pass

# Write Py3.12-friendly setup scripts that also compile ../cpp_utils/*.cpp
sub_setup = r"""
from setuptools import setup, Extension
import numpy as np
from pathlib import Path
def glob_srcs(root): return [str(p) for p in Path(root).rglob("*.cpp")]
sources = ["wrapper.cpp"] + glob_srcs("grid_subsampling") + glob_srcs("../cpp_utils")
ext = Extension("grid_subsampling", sources=sources,
                include_dirs=[np.get_include(), "grid_subsampling", "../cpp_utils"],
                language="c++", extra_compile_args=["-O3","-std=c++14"])
setup(name="grid_subsampling", version="0.0.0", ext_modules=[ext])
"""
nei_setup = r"""
from setuptools import setup, Extension
import numpy as np
from pathlib import Path
def glob_srcs(root): return [str(p) for p in Path(root).rglob("*.cpp")]
sources = ["wrapper.cpp"] + glob_srcs("neighbors") + glob_srcs("../cpp_utils")
ext = Extension("radius_neighbors", sources=sources,
                include_dirs=[np.get_include(), "neighbors", "../cpp_utils"],
                language="c++", extra_compile_args=["-O3","-std=c++14"])
setup(name="radius_neighbors", version="0.0.0", ext_modules=[ext])
"""
open("/content/KPConv-PyTorch/cpp_wrappers/cpp_subsampling/setup_py312.py","w").write(sub_setup)
open("/content/KPConv-PyTorch/cpp_wrappers/cpp_neighbors/setup_py312.py","w").write(nei_setup)

# Build in place
%cd /content/KPConv-PyTorch/cpp_wrappers/cpp_subsampling
!python3 setup_py312.py build_ext --inplace -v
%cd /content/KPConv-PyTorch/cpp_wrappers/cpp_neighbors
!python3 setup_py312.py build_ext --inplace -v

# Copy .so into YOUR project and make packages importable
import os
os.makedirs(f"{PROJECT_ROOT}/cpp_wrappers/cpp_subsampling", exist_ok=True)
os.makedirs(f"{PROJECT_ROOT}/cpp_wrappers/cpp_neighbors",   exist_ok=True)
open(f"{PROJECT_ROOT}/cpp_wrappers/__init__.py","a").close()
open(f"{PROJECT_ROOT}/cpp_wrappers/cpp_subsampling/__init__.py","a").close()
open(f"{PROJECT_ROOT}/cpp_wrappers/cpp_neighbors/__init__.py","a").close()

!cp /content/KPConv-PyTorch/cpp_wrappers/cpp_subsampling/*.so "{PROJECT_ROOT}/cpp_wrappers/cpp_subsampling/"
!cp /content/KPConv-PyTorch/cpp_wrappers/cpp_neighbors/*.so   "{PROJECT_ROOT}/cpp_wrappers/cpp_neighbors/"


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
/content/KPConv-PyTorch
Already up to date.
/content/KPConv-PyTorch/cpp_wrappers/cpp_subsampling
running build_ext
building 'grid_subsampling' extension
x86_64-linux-gnu-g++ -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.12/dist-packages/numpy/core/include -Igrid_subsampling -I../cpp_utils -I/usr/include/python3.12 -c ../cpp_utils/cloud/cloud.cpp -o build/temp.linux-x86_64-cpython-312/../cpp_utils/cloud/cloud.o -O3 -std=c++14
x86_64-linux-gnu-g++ -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.12/dist-packages/numpy/core/include -Igrid_subsampling -I../cpp_utils -I/usr/

In [ ]:
# Build KPConv wrappers in /content
!git clone --depth 1 https://github.com/HuguesTHOMAS/KPConv-PyTorch.git /content/KPConv-PyTorch || true
%cd /content/KPConv-PyTorch/cpp_wrappers/cpp_subsampling
!python3 setup.py build_ext --inplace -v
%cd /content/KPConv-PyTorch/cpp_wrappers/cpp_neighbors
!python3 setup.py build_ext --inplace -v
%cd /content
!ls -la /content/KPConv-PyTorch/cpp_wrappers/cpp_subsampling/*.so
!ls -la /content/KPConv-PyTorch/cpp_wrappers/cpp_neighbors/*.so

# Copy built wrappers into YOUR project so the import path matches
import os, shutil
dst = os.path.join(PROJECT_ROOT, "cpp_wrappers")
os.makedirs(dst, exist_ok=True)
!rsync -a /content/KPConv-PyTorch/cpp_wrappers/ "$dst"/


fatal: destination path '/content/KPConv-PyTorch' already exists and is not an empty directory.
/content/KPConv-PyTorch/cpp_wrappers/cpp_subsampling
Traceback (most recent call last):
  File "/content/KPConv-PyTorch/cpp_wrappers/cpp_subsampling/setup.py", line 2, in <module>
    import numpy.distutils.misc_util
ModuleNotFoundError: No module named 'numpy.distutils'
/content/KPConv-PyTorch/cpp_wrappers/cpp_neighbors
Traceback (most recent call last):
  File "/content/KPConv-PyTorch/cpp_wrappers/cpp_neighbors/setup.py", line 2, in <module>
    import numpy.distutils.misc_util
ModuleNotFoundError: No module named 'numpy.distutils'
/content
-rwxr-xr-x 1 root root 879168 Aug 21 00:15 /content/KPConv-PyTorch/cpp_wrappers/cpp_subsampling/grid_subsampling.cpython-312-x86_64-linux-gnu.so
-rwxr-xr-x 1 root root 687200 Aug 21 00:15 /content/KPConv-PyTorch/cpp_wrappers/cpp_neighbors/radius_neighbors.cpython-312-x86_64-linux-gnu.so


In [ ]:
import re, io, os
abc_path = os.path.join(PROJECT_ROOT, "datasets", "ABC.py")
with open(abc_path, "r", encoding="utf-8") as f:
    src = f.read()

src2 = re.sub(
    r"import cpp_wrappers\.cpp_subsampling\.grid_subsampling as _gs\s*?\n\s*?grid_subsample\s*=\s*_gs\.subsample\s*?\n\s*?grid_subsample_batch\s*=\s*_gs\.subsample_batch",
    """try:
    from cpp_wrappers.cpp_subsampling.grid_subsampling import subsample as grid_subsample
    from cpp_wrappers.cpp_subsampling.grid_subsampling import subsample_batch as grid_subsample_batch
except (ImportError, AttributeError):
    from cpp_wrappers.cpp_subsampling.grid_subsampling import grid_subsampling as grid_subsample
    def grid_subsample_batch(points, features=None, labels=None, sampleDl=0.1, verbose=0):
        return grid_subsample(points, features, labels, sampleDl, verbose)""",
    src, flags=re.DOTALL
)

if src2 != src:
    with open(abc_path, "w", encoding="utf-8") as f:
        f.write(src2)
    print("Patched ABC.py to support both subsample APIs.")
else:
    print("ABC.py already robust.")


ABC.py already robust.


In [ ]:
# Remove packages that drag in NumPy>=2
%pip -q uninstall -y numpy scipy scikit-learn tsfresh opencv-python opencv-contrib-python opencv-python-headless thinc umap-learn cuml-cu12 || true

# Install a consistent Py3.12 stack with NumPy 1.26
%pip -q install --no-cache-dir "numpy==1.26.4" "scipy==1.11.4" "scikit-learn==1.4.2" "pybind11>=2.10,<2.12" plyfile

# HARD RESTART so 1.26 is actually imported
import os; os.kill(os.getpid(), 9)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, which is not installed.
spacy 3.8.7 requires thinc<8.4.0,>=8.3.4, which is not installed.
dopamine-rl 4.1.2 requires opencv-python>=3.4.8.29, which is not installed.


In [ ]:
%pip -q install -U plyfile


Start training

In [ ]:
import importlib
from utils.config import Config
from utils.trainer import set_seed

cfg = Config()

set_seed(getattr(cfg, "seed", 42))

import os
os.environ["BATCH_SIZE"] = str(getattr(cfg, "batch_size", 2))
os.environ["NUM_WORKERS"] = str(getattr(cfg, "num_workers", 0))
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("Batch:", os.environ["BATCH_SIZE"], "Workers:", os.environ["NUM_WORKERS"])


[seed] Global seed set to 42
Batch: 2 Workers: 0


In [ ]:
from utils.config import Config
from utils.trainer import Trainer, set_seed
from datasets.ABC import ABCDataset, abc_collate
from models.architecture import build_from_cfg
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Load config
cfg = Config()
set_seed(cfg.seed)

# Ensure radii
if getattr(cfg, "layer_radii", None) is None:
    cfg.layer_radii = [
        cfg.conv_radius * cfg.first_subsampling_dl * float(m)
        for m in cfg.layer_multipliers
    ]

# Datasets
train_set = ABCDataset(cfg, split="Train",      use_ssm=cfg.use_ssm, use_normals=cfg.use_normals, pool_cap=cfg.pool_cap)
val_set   = ABCDataset(cfg, split="Validation", use_ssm=cfg.use_ssm, use_normals=cfg.use_normals, pool_cap=cfg.pool_cap)

# In-dim + model
in_dim = train_set[0]["features"][0].shape[1]
model = build_from_cfg(cfg, in_dim)

# Trainer
trainer = Trainer(cfg, model, train_set, val_set, device=device)
trainer.in_dim = in_dim  # stored in ckpts for tester convenience
trainer.fit()


Device: cuda
[seed] Global seed set to 42
[ABC] Fitting SSM mean/std on Train split (once) ...
Training for 60 epochs on device=cuda
[ep 01] train 0.5062 | val 0.1561 | P 0.462 R 0.355 F1 0.401 IoU 0.251 MCC 0.373 | peak 5.97 GB | 1565.1s
 ↑ Saved BEST (F1=0.401)
[ep 02] train 0.3194 | val 0.2059 | P 0.350 R 0.653 F1 0.456 IoU 0.295 MCC 0.435 | peak 5.71 GB | 136.7s
 ↑ Saved BEST (F1=0.456)
[ep 03] train 0.2866 | val 0.1844 | P 0.395 R 0.240 F1 0.299 IoU 0.176 MCC 0.276 | peak 5.82 GB | 133.1s
[ep 04] train 0.2370 | val 0.1389 | P 0.690 R 0.305 F1 0.423 IoU 0.268 MCC 0.438 | peak 5.88 GB | 134.0s
[ep 05] train 0.2337 | val 0.2730 | P 0.582 R 0.475 F1 0.523 IoU 0.354 MCC 0.500 | peak 6.00 GB | 133.3s
 ↑ Saved BEST (F1=0.523)
[ep 06] train 0.2007 | val 0.2016 | P 0.456 R 0.332 F1 0.384 IoU 0.238 MCC 0.357 | peak 6.08 GB | 133.5s
[ep 07] train 0.2191 | val 0.1554 | P 0.547 R 0.376 F1 0.446 IoU 0.287 MCC 0.426 | peak 5.92 GB | 134.1s
[ep 08] train 0.1820 | val 0.1593 | P 0.608 R 0.332 F1 0

In [ ]:
RUN_DIR = trainer.run_dir
print("Run dir:", RUN_DIR)

Run dir: /content/drive/MyDrive/kpconv_abc/runs/exp_default


testing

In [ ]:
from utils.tester import test_run

stats = test_run(
    run_dir=RUN_DIR,
    ckpt_name="model_best.pt",
    split="Validation",
    threshold=getattr(cfg, "default_threshold", 0.25),
    export_pred_ply=False,   # True if raw pred PLYs
    export_diff_ply=True,
    save_plots=True
)
print("Validation stats:", stats)


[tester] Validation: F1=0.919 P=0.876 R=0.967 IoU=0.850 MCC=0.915
Validation stats: {'precision': 0.8756972831364558, 'recall': 0.9672229289829888, 'f1': 0.9191873691200243, 'iou': 0.8504595003618791, 'mcc': 0.915221490862917, 'accuracy': 0.9901218612858951, 'specificity': 0.9915338967147689, 'tp': 38775, 'tn': 644618, 'fp': 5504, 'fn': 1314, 'threshold': 0.25, 'split': 'Validation', 'elapsed_sec': 19.148773908615112}
